# FEM-BEM coupling 

In [ ]:
from netgen.csg import *
from ngsolve import *
from ngsolve.webgui import Draw

In [ ]:
ball = Sphere(Pnt(0,0,0), 1)
geo = CSGeometry()
geo.Add(ball)
mesh = Mesh(geo.GenerateMesh(maxh=0.2))

Draw (mesh)

In [ ]:
order=1
V = H1(mesh,order=order)
Q = SurfaceL2(mesh, order=order-1)
X = V*Q
(u,lam), (v,mu) = X.TnT()

In [ ]:
f = LinearForm(X)
f += v(0,0,0)  # point source
f.Assemble()

a = BilinearForm(grad(u)*grad(v)*dx).Assemble()
b = BilinearForm(grad(u)*grad(v)*dx+u*v*dx + lam*mu*ds).Assemble()
inv = b.mat.Inverse()

gf = GridFunction(X)
gf.vec.data = inv * f.vec
gfu, gflam = gf.components

In [ ]:
Draw (gfu)

some stuff copied from example.py

In [ ]:
import ngbem
import bempp.api;
import numpy as np;
import scipy;
import bempp.core

# bempp.api.DEFAULT_DEVICE_INTERFACE="opencl"
bempp.api.DEFAULT_DEVICE_INTERFACE="numba"

In [ ]:
[bem_c,trace_matrix]=ngbem.H1_trace(V);
bem_dc=bempp.api.function_space(bem_c.grid,'DP',order-1);

In [ ]:
from bempp.api.assembly.blocked_operator import BlockedDiscreteOperator
from bempp.api.assembly.discrete_boundary_operator import InverseSparseDiscreteBoundaryOperator
##set up the bem
bempp.api.VECTORIZATION_MODE = "novec" 
sl=bempp.api.operators.boundary.laplace.single_layer(bem_dc,bem_c,bem_dc) # ,assembler="fmm", device_interface="opencl")
dl=bempp.api.operators.boundary.laplace.double_layer(bem_c,bem_c,bem_dc)#,assembler="fmm", device_interface="opencl")
id_op=bempp.api.operators.boundary.sparse.identity(bem_dc,bem_dc,bem_c)
id_op2=bempp.api.operators.boundary.sparse.identity(bem_c,bem_c,bem_dc)

In [ ]:
idmat = id_op.weak_form().A

print (type(idmat))
print (type(trace_matrix))
idmat.shape
print(X.Range(0), X.Range(1))

In [ ]:
# print(-trace_matrix.T * id_op.weak_form().A)

In [ ]:
print (sl.domain.global_dof_count)
print (sl.range.global_dof_count)

In [ ]:
op = sl.weak_form()
h,w = op.shape
print ("h=",h,"w=",w)

In [ ]:
vx = BaseVector(w)
vy = BaseVector(h)
vx[:] = 1
vy.FV().NumPy()[:] = op * vx

In [ ]:
print (vy)

In [ ]:
class NGSOperator(BaseMatrix):
    def __init__(self, mat):
        BaseMatrix.__init__(self)
        self.mat = mat
    def IsComplex(self):
        return False
    def Height(self):
        return self.mat.shape[0]
    def Width(self):
        return self.mat.shape[1]
    def CreateRowVector(self):
        return BaseVector(self.Width())
    def CreateColVector(self):
        return BaseVector(self.Height())
    def Mult(self,x,y):
        y.FV().NumPy()[:] = self.mat * x    

In [ ]:
ngsop = NGSOperator(op)
x = ngsop.CreateRowVector()
y = ngsop.CreateColVector()
x[:] = 1
y.data = ngsop * x
print(y)

The product space can provide embedding matrices for the individual components:

$$
E_u = \left( \begin{array}{c} I \\ 0 \end{array} \right)
\qquad
E_\lambda = \left( \begin{array}{c} 0 \\ I \end{array} \right)
$$


In [ ]:
embu, emblam = X.embeddings

In [ ]:
ngs_dl = NGSOperator(dl.weak_form())
ngs_dl = emblam @ ngs_dl @ emblam.T   # 1,1 block

In [ ]:
len(ngs_dl.CreateRowVector())